In [1]:
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

In [2]:
df = pd.read_csv("2.4.csv")
athletes = df.columns
sports = pd.Index(("Sport1", "Sport2", "Sport3", "Sport4"))
df.index = sports
df

,Athlete1,Athlete2,Athlete3,Athlete4,Athlete5,Athlete6,Athlete7,Athlete8,Athlete9,Athlete10
Sport1,8.4~0.15 9.5~0.5 9.2~0.25 9.4~0.1,9.3~0.1 9.5~0.1 9.6~0.6 9.8~0.2,8.4~0.1 8.8~0.2 9.0~0.6 10~0.1,8.1~0.1 9.1~0.5 9.3~0.3 9.5~0.1,8.4~0.15 9.5~0.5 9.2~0.25 9.4~0.1,9.4~0.1 9.6~0.1 9.7~0.6 9.9~0.2,9.5~0.1 9.7~0.1 9.8~0.6 10~0.2,8.4~0.1 8.8~0.2 9.0~0.6 10~0.1,8.4~0.15 9.5~0.5 9.2~0.25 9.4~0.1,9.0~0.1 9.2~0.1 9.4~0.6 9.7~0.2
Sport2,8.4~0.1 8.8~0.2 9.0~0.6 10~0.1,8.4~0.15 9.0~0.5 9.2~0.25 9.4~0.1,8.1~0.1 9.1~0.5 9.3~0.3 9.5~0.1,8.7~0.1 8.9~0.2 9.1~0.6 9.9~0.1,9.0~0.1 9.2~0.1 9.4~0.6 9.7~0.2,8.7~0.1 8.9~0.2 9.1~0.6 9.9~0.1,8.4~0.1 8.8~0.2 9.0~0.6 10~0.1,8.8~0.05 9.2~0.05 9.8~0.5 10~0.4,8.4~0.1 8.8~0.1 9.2~0.6 9.8~0.2,8.1~0.1 9.1~0.5 9.3~0.3 9.5~0.1
Sport3,9.1~0.1 9.3~0.1 9.5~0.6 9.8~0.2,8.4~0.1 8.8~0.2 9.0~0.6 10~0.1,8.4~0.15 9.5~0.5 9.2~0.25 9.4~0.1,9.0~0.1 9.4~0.1 9.5~0.5 9.7~0.3,8.3~0.1 8.7~0.1 8.9~0.6 9.3~0.2,8.5~0.1 8.7~0.1 8.9~0.5 9.1~0.3,8.3~0.1 8.7~0.1 8.9~0.6 9.3~0.2,8.7~0.1 8.9~0.2 9.1~0.6 9.9~0.1,8.4~0.1 8.8~0.2 9.0~0.6 10~0.1,8.2~0.1 9.2~0.5 9.4~0.3 9.6~0.1
Sport4,8.7~0.1 8.9~0.2 9.1~0.6 9.9~0.1,8.9~0.1 9.1~0.1 9.3~0.6 9.6~0.2,9.5~0.1 9.7~0.1 9.8~0.6 10~0.2,8.4~0.1 8.8~0.2 9.0~0.6 10~0.1,9.4~0.1 9.6~0.1 9.7~0.6 9.9~0.2,8.4~0.15 9.5~0.5 9.2~0.25 9.4~0.1,8.4~0.1 8.8~0.1 9.2~0.6 9.8~0.2,8.2~0.1 9.3~0.5 9.5~0.3 9.8~0.1,9.3~0.1 9.5~0.1 9.7~0.5 9.9~0.3,9.1~0.1 9.3~0.1 9.5~0.6 9.8~0.2


In [3]:
df_long = pd.melt(
    df.reset_index(), id_vars="index", var_name="运动员", value_name="得分~概率"
)
df_long.columns = ["项目", "运动员", "得分~概率"]
df_long.head()

,项目,运动员,得分~概率
0,Sport1,Athlete1,8.4~0.15 9.5~0.5 9.2~0.25 9.4~0.1
1,Sport2,Athlete1,8.4~0.1 8.8~0.2 9.0~0.6 10~0.1
2,Sport3,Athlete1,9.1~0.1 9.3~0.1 9.5~0.6 9.8~0.2
3,Sport4,Athlete1,8.7~0.1 8.9~0.2 9.1~0.6 9.9~0.1
4,Sport1,Athlete2,9.3~0.1 9.5~0.1 9.6~0.6 9.8~0.2


In [4]:
from decimal import Decimal


expanded_data = []

for i, row in df_long.iterrows():
    for item in row["得分~概率"].split(" "):
        score, prob = map(Decimal, item.split("~"))
        expanded_data.append(
            {"项目": row["项目"], "运动员": row["运动员"], "得分": score, "概率": prob}
        )
expanded_df = pd.DataFrame(expanded_data)
expanded_df.head()

,项目,运动员,得分,概率
0,Sport1,Athlete1,8.4,0.15
1,Sport1,Athlete1,9.5,0.5
2,Sport1,Athlete1,9.2,0.25
3,Sport1,Athlete1,9.4,0.1
4,Sport2,Athlete1,8.4,0.1


In [5]:
# 计算最低得分和期望平均分
summary_stats = (
    expanded_df.groupby(["项目", "运动员"])
    .apply(
        lambda x: pd.Series(
            {"最低得分": x["得分"].min(), "期望平均分": (x["得分"] * x["概率"]).sum()}
        )
    )
    .reset_index()
)
summary_stats.head()

/var/folders/n_/d82sr3dd4jgfqk4vk5sblcn80000gn/T/ipykernel_89197/3987200986.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(


,项目,运动员,最低得分,期望平均分
0,Sport1,Athlete1,8.4,9.250
1,Sport1,Athlete10,9.0,9.40
2,Sport1,Athlete2,9.3,9.60
3,Sport1,Athlete3,8.4,9.00
4,Sport1,Athlete4,8.1,9.10


In [6]:
combos, low, mean = gp.multidict(
    {
        (row["项目"], row["运动员"]): [row["最低得分"], row["期望平均分"]]
        for _, row in summary_stats.iterrows()
    }
)

In [7]:
m = gp.Model()

In [8]:
x = m.addVars(combos, vtype=GRB.BINARY, name="x")

In [9]:
# y_i==1: 第j人参加全能比赛
y = m.addVars(athletes, vtype=GRB.BINARY, name="y")

## 选手得分按最悲观估算

In [10]:
m.setObjective(x.prod(low), sense=GRB.MAXIMIZE)

In [11]:
athletes

Index(['Athlete1', 'Athlete2', 'Athlete3', 'Athlete4', 'Athlete5', 'Athlete6',
       'Athlete7', 'Athlete8', 'Athlete9', 'Athlete10'],
      dtype='object')

In [12]:
m.addConstrs(
    (x.sum("*", athlete) <= 3 + y[athlete] for athlete in athletes),
    name="是否参加全能比赛上限约束",
)
m.addConstrs(
    (4 * y[athlete] <= x.sum("*", athlete) for athlete in athletes),
    name="是否参加全能比赛下限约束",
)

{'Athlete1': <gurobi.Constr *Awaiting Model Update*>,
 'Athlete2': <gurobi.Constr *Awaiting Model Update*>,
 'Athlete3': <gurobi.Constr *Awaiting Model Update*>,
 'Athlete4': <gurobi.Constr *Awaiting Model Update*>,
 'Athlete5': <gurobi.Constr *Awaiting Model Update*>,
 'Athlete6': <gurobi.Constr *Awaiting Model Update*>,
 'Athlete7': <gurobi.Constr *Awaiting Model Update*>,
 'Athlete8': <gurobi.Constr *Awaiting Model Update*>,
 'Athlete9': <gurobi.Constr *Awaiting Model Update*>,
 'Athlete10': <gurobi.Constr *Awaiting Model Update*>}

In [13]:
m.addConstrs((x.sum(sport, "*") == 6 for sport in sports), name="每个项目最多出场六人")

{'Sport1': <gurobi.Constr *Awaiting Model Update*>,
 'Sport2': <gurobi.Constr *Awaiting Model Update*>,
 'Sport3': <gurobi.Constr *Awaiting Model Update*>,
 'Sport4': <gurobi.Constr *Awaiting Model Update*>}

In [14]:
m.addConstr(y.sum() == 4, name="应有四人参加全能比赛")

<gurobi.Constr *Awaiting Model Update*>

In [15]:
m.optimize()

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (mac64[arm] - Darwin 23.4.0 23E224)

CPU model: Apple M1 Pro
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 25 rows, 50 columns and 150 nonzeros
Model fingerprint: 0x0ee617dc
Variable types: 0 continuous, 50 integer (50 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [8e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 6e+00]
Found heuristic solution: objective 208.2000000
Presolve time: 0.00s
Presolved: 25 rows, 50 columns, 150 nonzeros
Variable types: 0 continuous, 50 integer (50 binary)
Found heuristic solution: objective 211.7000000

Root relaxation: objective 2.130000e+02, 16 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  213.00000    0    7  211.70000  213.00000 

In [16]:
print(f"Objective value: {m.ObjVal}")
print("Optimal solution:")
for i in m.getVars():
    if i.X > 1e-6:
        print(f"{i.VarName} = {i.X}", end="\n")

Objective value: 212.30000000000004
Optimal solution:
x[Sport1,Athlete10] = 1.0
x[Sport1,Athlete2] = 1.0
x[Sport1,Athlete5] = 1.0
x[Sport1,Athlete6] = 1.0
x[Sport1,Athlete7] = 1.0
x[Sport1,Athlete9] = 1.0
x[Sport2,Athlete2] = 1.0
x[Sport2,Athlete4] = 1.0
x[Sport2,Athlete5] = 1.0
x[Sport2,Athlete6] = 1.0
x[Sport2,Athlete8] = 1.0
x[Sport2,Athlete9] = 1.0
x[Sport3,Athlete1] = 1.0
x[Sport3,Athlete2] = 1.0
x[Sport3,Athlete4] = 1.0
x[Sport3,Athlete5] = 1.0
x[Sport3,Athlete6] = 1.0
x[Sport3,Athlete9] = 1.0
x[Sport4,Athlete10] = 1.0
x[Sport4,Athlete2] = 1.0
x[Sport4,Athlete3] = 1.0
x[Sport4,Athlete5] = 1.0
x[Sport4,Athlete6] = 1.0
x[Sport4,Athlete9] = 1.0
y[Athlete2] = 1.0
y[Athlete5] = 1.0
y[Athlete6] = 1.0
y[Athlete9] = 1.0


## 选手得分按均值估算

In [17]:
m.setObjective(x.prod(mean), sense=GRB.MAXIMIZE)

In [18]:
m.optimize()

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (mac64[arm] - Darwin 23.4.0 23E224)

CPU model: Apple M1 Pro
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 25 rows, 50 columns and 150 nonzeros
Model fingerprint: 0x64adea5d
Variable types: 0 continuous, 50 integer (50 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [9e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 6e+00]

Loaded MIP start from previous solve with objective 224.65

Presolve time: 0.00s
Presolved: 25 rows, 50 columns, 150 nonzeros
Variable types: 0 continuous, 50 integer (50 binary)

Root relaxation: objective 2.255500e+02, 13 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  225.55000    0    6  224.65000  225.55000  0.40%     -    0s
H    0     0    

In [19]:
print(f"Objective value: {m.ObjVal}")
print("Optimal solution:")
for i in m.getVars():
    if i.X > 1e-6:
        print(f"{i.VarName} = {i.X}", end="\n")

Objective value: 225.1
Optimal solution:
x[Sport1,Athlete10] = 1.0
x[Sport1,Athlete2] = 1.0
x[Sport1,Athlete3] = 1.0
x[Sport1,Athlete6] = 1.0
x[Sport1,Athlete7] = 1.0
x[Sport1,Athlete9] = 1.0
x[Sport2,Athlete10] = 1.0
x[Sport2,Athlete2] = 1.0
x[Sport2,Athlete3] = 1.0
x[Sport2,Athlete5] = 1.0
x[Sport2,Athlete8] = 1.0
x[Sport2,Athlete9] = 1.0
x[Sport3,Athlete1] = 1.0
x[Sport3,Athlete10] = 1.0
x[Sport3,Athlete2] = 1.0
x[Sport3,Athlete3] = 1.0
x[Sport3,Athlete4] = 1.0
x[Sport3,Athlete9] = 1.0
x[Sport4,Athlete10] = 1.0
x[Sport4,Athlete2] = 1.0
x[Sport4,Athlete3] = 1.0
x[Sport4,Athlete5] = 1.0
x[Sport4,Athlete8] = 1.0
x[Sport4,Athlete9] = 1.0
y[Athlete2] = 1.0
y[Athlete3] = 1.0
y[Athlete9] = 1.0
y[Athlete10] = 1.0
